In [ ]:
## Create an extension to the visualizations module that adds a new type of plot for finding the common earthquakes.


In [ ]:
# 📦 Imports
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 📂 Load GeoJSON
with open("data/GeoQuake.json") as f:
    data = json.load(f)

# 📊 Parse into DataFrame
records = []
for feature in data["features"]:
    props = feature.get("properties", {})
    coords = feature.get("geometry", {}).get("coordinates", [None, None, None])
    products = props.get("products", {})

    try:
        record = {
            "time": pd.to_datetime(props.get("time"), unit="ms", errors="coerce"),
            "updated": pd.to_datetime(props.get("updated"), unit="ms", errors="coerce"),
            "latitude": round(coords[1], 1) if coords[1] is not None else None,
            "longitude": round(coords[0], 1) if coords[0] is not None else None,
            "depth": coords[2],
            "mag": props.get("mag"),
            "felt": props.get("felt"),
            "cdi": props.get("cdi"),
            "mmi": props.get("mmi"),
            "rms": props.get("rms"),
            "gap": props.get("gap"),
            "dmin": props.get("dmin"),
            "nst": props.get("nst"),
            "sig": props.get("sig"),
            "tsunami": props.get("tsunami"),
            "place": props.get("place"),
            "status": props.get("status"),
            "alert": props.get("alert"),
            "net": props.get("net"),
            "code": props.get("code"),
            "magType": props.get("magType"),
            "type": props.get("type"),
            "ids": props.get("ids"),
            "sources": props.get("sources"),
            "types": props.get("types"),
            "has_shakemap": "shakemap" in products,
            "has_moment_tensor": "moment-tensor" in products,
            "has_focal_mechanism": "focal-mechanism" in products
        }
        records.append(record)
    except Exception:
        continue

df = pd.DataFrame(records)
df = df.dropna(subset=["latitude", "longitude", "time"])
df["date"] = df["time"].dt.date
df["location_id"] = df["latitude"].astype(str) + "," + df["longitude"].astype(str)

# 🧱 Create daily transactions: list of locations per day
daily_events = df.groupby("date")["location_id"].apply(list).tolist()

# 🔄 Encode transactions
te = TransactionEncoder()
te_ary = te.fit(daily_events).transform(daily_events)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 📈 Mine frequent itemsets
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

# 🔗 Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules = rules[rules["antecedents"].apply(lambda x: len(x) == 1)]
rules = rules[rules["consequents"].apply(lambda x: len(x) == 1)]

# 🧹 Extract A → B pairs
def extract_coords(location_str):
    lat, lon = map(float, location_str.split(","))
    return lat, lon

rules["A"] = rules["antecedents"].apply(lambda x: list(x)[0])
rules["B"] = rules["consequents"].apply(lambda x: list(x)[0])
rules["A_lat"], rules["A_lon"] = zip(*rules["A"].map(extract_coords))
rules["B_lat"], rules["B_lon"] = zip(*rules["B"].map(extract_coords))

# 🌍 Plot on world map
fig = go.Figure()

for _, row in rules.iterrows():
    fig.add_trace(go.Scattergeo(
        lon=[row["A_lon"], row["B_lon"]],
        lat=[row["A_lat"], row["B_lat"]],
        mode="lines+markers",
        line=dict(width=1.5, color="red"),
        marker=dict(size=4),
        hoverinfo="text",
        text=f"{row['A']} → {row['B']}<br>Conf: {row['confidence']:.2f}, Supp: {row['support']:.2f}"
    ))

fig.update_layout(
    title="Frequent Earthquake Co-Occurrence Pairs (A → B)",
    geo=dict(
        projection_type="natural earth",
        showland=True,
        landcolor="rgb(217, 217, 217)",
        showcountries=True,
    ),
    height=600
)

fig.show()